## 목차

1. 프로젝트 개요  
    1.1. 문제 정의 및 목표  
    1.2. 데이터셋 구조 및 평가 방법  

2. 환경 설정 및 데이터 준비  
    2.1. 환경 설정 및 라이브러리 설치  
    2.2. 한–영 JSON 데이터 로드 및 기초 탐색  
    2.3. 토크나이징, 문장 길이 분석 및 MAX_LENGTH 설정  
    2.4. 특수 토큰 정의 및 어휘 사전 구축  

3. 번역 모델 구현(Seq2Seq, Seq2Seq + Attention)  
    3.1. 입력·출력 시퀀스 텐서 변환 및 DataLoader 구성  
    3.2. GRU 기반 Encoder–Decoder Seq2Seq 모델 정의  
    3.3. Seq2Seq 모델 학습 루프 정의  
    3.4. Seq2Seq + Attention(Bahdanau) 적용 디코더 구현  
    3.5. 각 모델 학습 및 번역 결과 확인  

4. 두 모델의 성능 평가 및 개선점 도출  
    4.1. Seq2Seq vs Attention 번역 품질 정성 평가  
    4.2. 정량 지표 비교 및 오번역 사례 분석  
    4.3. 개선점 정리  

5. 추가 실험  
    5.1. 모델 구조 및 하이퍼파라미터 변경 실험  
    5.2. 학습 설정 변경 실험  
    5.3. 전처리 방법 변경 실험  
    5.4. 가장 효과적이었던 설정 요약  

6. 결론  
    6.1. 전체 실험 결과 요약 및 인사이트  
    6.2. 한계점 및 향후 개선 방향  

## 1. 프로젝트 개요

### 1.1. 문제 정의 및 목표

- m 이 프로젝트의 목표는 한국어 문장을 영어로 번역하는 기계 번역 모델을 직접 구현하고 비교하는 것이다.
​
- JSON 형식의 한\–영 병렬 말뭉치를 사용해 Seq2Seq 기본 모델과 Attention 기반 모델을 학습시키고, 번역 예시와 간단한 지표를 통해 성능 차이를 분석한다.

### 1.2. 데이터셋 구조 및 평가 방법

- 데이터는 각 샘플이 {"ko": 한국어 문장, "mt": 영어 번역문} 형태인 JSON 리스트로 제공되며, train/valid 파일이 분리되어 있다.
​
- 모델 평가는 무작위 문장 쌍에 대한 번역 결과를 직접 확인하는 정성 평가를 중심으로 수행하고, BLEU 와 같은 정량 지표를 보조적으로 활용한다.

## 2. 환경 설정 및 데이터 준비  

### 2.1. 환경 설정 및 라이브러리 설치

- 기계 번역 실험에 필요한 파이썬 라이브러리와 GPU 환경을 설정한다.
​
- 로컬 환경에서 JSON 데이터 로드, 한국어·영어 토크나이징, PyTorch 기반 Seq2Seq/Attention 모델 학습이 가능하도록 패키지를 불러오고 디바이스를 지정한다.

In [1]:
import os
import json
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

from konlpy.tag import Okt
import nltk
from nltk.tokenize import word_tokenize

import torch.nn.functional as F

# NLTK 리소스 다운로드
nltk.download('punkt')
nltk.download('punkt_tab')

# 디바이스 설정 (GPU 사용 가능 시 cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


[nltk_data] Downloading package punkt to /home/kmw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/kmw/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 2.2. 한–영 JSON 데이터 로드 및 기초 탐색

- 이 섹션에서는 JSON 형식으로 제공된 한국어–영어 병렬 코퍼스를 로드하고, 문장 쌍의 개수와 예시를 간단히 확인해 데이터 구조를 파악한다.
​
- 각 샘플은 {"ko": 한국어 문장, "mt": 영어 번역문} 형태로 저장되어 있으며, 학습용(train)과 검증용(valid) 파일을 분리해서 읽는다.

In [2]:
# 데이터 경로 설정 (환경에 맞게 수정)
base_path = "/home/kmw/github/mission11"
train_json_file_path = os.path.join(base_path, "dataset", "일상생활및구어체_한영", "일상생활및구어체_한영_train_set.json")
valid_json_file_path = os.path.join(base_path, "dataset", "일상생활및구어체_한영", "일상생활및구어체_한영_valid_set.json")

# JSON 로드 함수
def load_json(file_path, max_samples=None):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data_list = data["data"]
    if max_samples is not None:
        data_list = data_list[:max_samples]
    return data_list

# 훈련·검증 데이터 로드 (필요시 max_samples로 샘플 수 제한)
data_train = load_json(train_json_file_path, max_samples=50000)
data_valid = load_json(valid_json_file_path, max_samples=1000)

# ko / mt 문장 리스트 분리
ko_sentences_train = [item["ko"] for item in data_train]
mt_sentences_train = [item["mt"] for item in data_train]
ko_sentences_valid = [item["ko"] for item in data_valid]
mt_sentences_valid = [item["mt"] for item in data_valid]

print("Train size:", len(ko_sentences_train))
print("Valid size:", len(ko_sentences_valid))
print("Sample pair:")
print("KO:", ko_sentences_train[0])
print("EN:", mt_sentences_train[0])

Train size: 50000
Valid size: 1000
Sample pair:
KO: 원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.
EN: If you reply to the color you want, we will start making it right away.


### 2.3. 토크나이징, 문장 길이 분석 및 MAX_LENGTH 설정

- 한국어·영어 문장을 토큰 단위로 분해하고, 길이 분포를 확인해 Seq2Seq 모델에 사용할 최대 길이(MAX_LENGTH)를 정한다.
​
- 베이스 코드에서는 한국어는 Okt 형태소 분석기, 영어는 NLTK word_tokenize를 사용하며, 한국어·영어 전체에서 가장 긴 문장을 기준으로 MAX_LENGTH를 잡는다.

In [3]:
# 토크나이저 정의
tokenizer_ko = Okt().morphs      # 한국어: 형태소 단위 토크나이징
tokenizer_en = word_tokenize     # 영어: 단어 단위 토크나이징

# 각 문장의 토큰 길이 계산
ko_lengths = [len(tokenizer_ko(sent)) for sent in ko_sentences_train]
en_lengths = [len(tokenizer_en(sent)) for sent in mt_sentences_train]
all_lengths = ko_lengths + en_lengths

# 한국어/영어 중 가장 긴 문장을 기준으로 MAX_LENGTH 설정 (SOS/EOS 고려 +1)
MAX_LENGTH = max(max(ko_lengths), max(en_lengths)) + 1
print(f"Max sequence length: {MAX_LENGTH}")

Max sequence length: 96


### 2.4. 특수 토큰 정의 및 어휘 사전 구축

- Seq2Seq 모델이 문장 시작·끝·패딩을 구분할 수 있도록 특수 토큰을 정의하고, 한국어·영어 각각에 대해 단어→인덱스 사전을 구축한다.
​
- 베이스 코드에서는 PAD, SOS, EOS, UNK를 미리 예약해 두고, 훈련 데이터의 모든 문장을 돌면서 어휘 사전을 확장한다.

In [4]:
# 특수 토큰 인덱스 정의
SOS_token = 0
EOS_token = 1
PAD_token = 2
UNK_token = 3

class Lang:
    def __init__(self, name):
        self.name = name
        # 기본 특수 토큰 4개 등록
        self.word2index = {
            PAD_token: "PAD",
            SOS_token: "SOS",
            EOS_token: "EOS",
            UNK_token: "<unk>"
        }
        self.index2word = {
            PAD_token: "PAD",
            SOS_token: "SOS",
            EOS_token: "EOS",
            UNK_token: "<unk>"
        }
        self.word2count = {}
        self.n_words = 4  # 특수 토큰 4개 포함

    def addSentence(self, sentence, tokenizer):
        for word in tokenizer(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

def prepareData(lang1, lang2, tokenizer1, tokenizer2):
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    pairs = list(zip(ko_sentences_train, mt_sentences_train))
    print("Read %s sentence pairs" % len(pairs))
    for ko, en in pairs:
        input_lang.addSentence(ko, tokenizer1)
        output_lang.addSentence(en, tokenizer2)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData("ko", "en", tokenizer_ko, tokenizer_en)

Read 50000 sentence pairs


## 3. 번역 모델 구현(Seq2Seq, Seq2Seq + Attention)  


### 3.1. 입력·출력 시퀀스 텐서 변환 및 DataLoader 구성

- 한국어·영어 문장을 어휘 사전의 인덱스 시퀀스로 변환하고, MAX_LENGTH에 맞춰 패딩한 뒤, PyTorch DataLoader로 학습 배치를 구성한다.
​
- 특히 SOS/EOS 토큰을 앞·뒤에 붙여 문장의 시작과 끝을 명시하고, 부족한 길이는 PAD 토큰으로 채워 고정 길이 시퀀스를 만든다.

In [5]:
def tensorFromSentence(lang, sentence, tokenizer):
    # 1) SOS 토큰으로 시작
    indexes = [SOS_token]
    # 2) 단어를 인덱스로 변환 (사전에 없으면 UNK 사용), MAX_LENGTH-2까지만 사용
    indexes += [
        lang.word2index.get(word, UNK_token)
        for word in tokenizer(sentence)[:MAX_LENGTH - 2]
    ]
    # 3) EOS 토큰 추가
    indexes.append(EOS_token)
    # 4) MAX_LENGTH까지 PAD로 채우기
    while len(indexes) < MAX_LENGTH:
        indexes.append(PAD_token)
    return torch.tensor(indexes[:MAX_LENGTH], dtype=torch.long, device=device)

In [6]:
def get_dataloader(batch_size):
    input_tensors = [tensorFromSentence(input_lang, ko, tokenizer_ko) for ko, _ in pairs]
    target_tensors = [tensorFromSentence(output_lang, en, tokenizer_en) for _, en in pairs]

    input_tensors = torch.stack(input_tensors, dim=0)   # [num_samples, MAX_LENGTH]
    target_tensors = torch.stack(target_tensors, dim=0) # [num_samples, MAX_LENGTH]

    dataset = TensorDataset(input_tensors, target_tensors)
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)

    print(f"input_tensors.shape: {input_tensors.shape}, target_tensors.shape: {target_tensors.shape}")
    return train_dataloader

In [7]:
# 검증 데이터를 위한 pair 생성
pairs_valid = list(zip(ko_sentences_valid, mt_sentences_valid))
print("Read %s validation sentence pairs" % len(pairs_valid))

Read 1000 validation sentence pairs


In [8]:
# 검증용 DataLoader 생성 함수
def get_valid_dataloader(batch_size):
    input_tensors = [tensorFromSentence(input_lang, ko, tokenizer_ko) for ko, _ in pairs_valid]
    target_tensors = [tensorFromSentence(output_lang, en, tokenizer_en) for _, en in pairs_valid]

    input_tensors = torch.stack(input_tensors, dim=0)   # [num_valid, MAX_LENGTH]
    target_tensors = torch.stack(target_tensors, dim=0) # [num_valid, MAX_LENGTH]

    dataset = TensorDataset(input_tensors, target_tensors)
    valid_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    print(
        f"[VALID] input_tensors.shape: {input_tensors.shape}, "
        f"target_tensors.shape: {target_tensors.shape}"
    )
    return valid_dataloader

In [9]:
# 학습/검증 DataLoader 생성
train_dataloader = get_dataloader(batch_size=32)
valid_dataloader = get_valid_dataloader(batch_size=32)

input_tensors.shape: torch.Size([50000, 96]), target_tensors.shape: torch.Size([50000, 96])
[VALID] input_tensors.shape: torch.Size([1000, 96]), target_tensors.shape: torch.Size([1000, 96])


### 3.2. GRU 기반 Encoder–Decoder Seq2Seq 모델 정의

- 한국어 입력 시퀀스를 인코더 GRU로 인코딩하고, 그 최종 은닉 상태를 디코더 GRU의 초기 상태로 넘겨 영어 번역 시퀀스를 생성하는 기본 Seq2Seq 구조를 구현한다.
​
- 인코더는 전체 입력 문장을 한 번에 처리해 문맥 벡터를 만들고, 디코더는 SOS 토큰에서 시작해 한 타임스텝씩 다음 단어를 예측하며, 학습 시에는 Teacher Forcing으로 정답 토큰을 다음 입력으로 사용하는 방식이다.

In [10]:
# Encoder 정의
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        # input: [batch_size, seq_len]
        embedded = self.dropout(self.embedding(input))    # [batch_size, seq_len, hidden_size]
        output, hidden = self.gru(embedded)               # hidden: [1, batch_size, hidden_size]
        return output, hidden

# Decoder 정의
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        # 초기 디코더 입력: SOS 토큰
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=encoder_outputs.device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: 정답 토큰을 다음 입력으로 사용
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                # 예측 토큰을 다음 입력으로 사용
                _, topi = decoder_output.topk(1)          # [batch, 1, 1]
                decoder_input = topi.squeeze(2).detach()  # [batch, 1]

        decoder_outputs = torch.cat(decoder_outputs, dim=1)   # [batch, MAX_LENGTH, output_size]
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None

    def forward_step(self, input, hidden):
        # input: [batch_size, 1]
        output = self.embedding(input)              # [batch_size, 1, hidden_size]
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)   # [batch_size, 1, hidden_size]
        output = self.out(output)                   # [batch_size, 1, output_size]
        return output, hidden

### 3.3. Seq2Seq 모델 학습 루프 정의

- 인코더·디코더를 한 번의 학습 스텝 안에서 연결하고, Teacher Forcing을 활용해 손실을 계산하는 학습 루프를 정의한다.
​
- 각 배치에서 인코더 출력과 디코더 출력 전체 시퀀스를 얻고, 디코더의 모든 타임스텝 예측을 정답 토큰 시퀀스와 비교해 NLLLoss로 학습 손실과 검증 손실을 동일한 방식(Teacher Forcing 기반)으로 계산한다.

In [11]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
                decoder_optimizer, criterion):
    encoder.train()
    decoder.train()

    total_loss = 0
    for input_tensor, target_tensor in dataloader:
        input_tensor = input_tensor.long().to(device)
        target_tensor = target_tensor.long().to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


def evaluate_epoch(dataloader, encoder, decoder, criterion):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    with torch.no_grad():
        for input_tensor, target_tensor in dataloader:
            input_tensor = input_tensor.long().to(device)
            target_tensor = target_tensor.long().to(device)

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            # 검증에서도 학습과 동일하게 Teacher Forcing 사용 (손실 비교용)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)
            )
            total_loss += loss.item()

    return total_loss / len(dataloader)


def train_seq2seq(train_dataloader, valid_dataloader,
                  encoder, decoder, n_epochs,
                  learning_rate=0.001, print_every=1):

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    best_val_loss = float("inf")

    for epoch in range(1, n_epochs + 1):
        train_loss = train_epoch(
            train_dataloader, encoder, decoder,
            encoder_optimizer, decoder_optimizer, criterion
        )
        val_loss = evaluate_epoch(
            valid_dataloader, encoder, decoder, criterion
        )

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # 필요하면 여기서 모델 가중치 저장 가능
            # torch.save({...}, "best_seq2seq.pt")

        if epoch % print_every == 0:
            print(
                f"Epoch {epoch}/{n_epochs} "
                f"| Train Loss: {train_loss:.4f} "
                f"| Val Loss: {val_loss:.4f}"
            )

### 3.4. Seq2Seq + Attention(Bahdanau) 적용 디코더 구현  

- 인코더가 만든 전체 hidden state 시퀀스를 참조해, 디코더가 각 타임스텝마다 “어떤 위치의 정보에 집중할지”를 학습하는 Bahdanau Attention 기반 디코더를 구현한다.
​
- 쿼리(query)는 디코더 hidden state, 키/값(keys, values)는 인코더 출력 전체 시퀀스로 두고, 어텐션 가중치로 가중합한 컨텍스트 벡터를 디코더 GRU 입력에 concat하여 번역 품질을 높인다.

In [12]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        # query: [batch, 1, hidden_size]
        # keys:  [batch, seq_len, hidden_size]
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        # scores: [batch, seq_len, 1] → [batch, 1, seq_len]
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)          # 어텐션 가중치
        context = torch.bmm(weights, keys)           # [batch, 1, hidden_size]
        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)  # Teacher forcing
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)   # [batch, MAX_LENGTH, vocab_size]
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)             # [batch, MAX_LENGTH, seq_len]

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))        # [batch, 1, hidden_size]

        query = hidden.permute(1, 0, 2)                       # [batch, 1, hidden_size]
        context, attn_weights = self.attention(query, encoder_outputs)
        # 임베딩 + 컨텍스트를 GRU 입력으로 결합
        input_gru = torch.cat((embedded, context), dim=2)     # [batch, 1, 2*hidden_size]

        output, hidden = self.gru(input_gru, hidden)          # [batch, 1, hidden_size]
        output = self.out(output)                             # [batch, 1, output_size]

        return output, hidden, attn_weights

### 3.5. 각 모델 학습 및 번역 결과 확인

- 두 모델을 각각 학습시키고, 임의 문장에 대한 번역 결과를 확인한다.
​
- 같은 DataLoader·에폭 수·학습률을 사용해 공정하게 비교하고, 학습 후에는 몇 개의 문장 쌍을 뽑아 원문·정답·모델 번역을 나란히 출력한다.

In [13]:
# 공통 하이퍼파라미터
hidden_size = 128
n_epochs = 30
learning_rate = 0.001

In [16]:
# 1) 순수 Seq2Seq 모델 학습
encoder_seq2seq = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_seq2seq = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train_seq2seq(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,   # 앞서 만든 검증용 DataLoader
    encoder=encoder_seq2seq,
    decoder=decoder_seq2seq,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    print_every=1
)

Epoch 1/30 | Train Loss: 0.7805 | Val Loss: 0.6199
Epoch 2/30 | Train Loss: 0.5769 | Val Loss: 0.5692
Epoch 3/30 | Train Loss: 0.5271 | Val Loss: 0.5454
Epoch 4/30 | Train Loss: 0.4926 | Val Loss: 0.5322
Epoch 5/30 | Train Loss: 0.4655 | Val Loss: 0.5250
Epoch 6/30 | Train Loss: 0.4432 | Val Loss: 0.5224
Epoch 7/30 | Train Loss: 0.4239 | Val Loss: 0.5195
Epoch 8/30 | Train Loss: 0.4069 | Val Loss: 0.5188
Epoch 9/30 | Train Loss: 0.3917 | Val Loss: 0.5208
Epoch 10/30 | Train Loss: 0.3783 | Val Loss: 0.5225
Epoch 11/30 | Train Loss: 0.3662 | Val Loss: 0.5250
Epoch 12/30 | Train Loss: 0.3553 | Val Loss: 0.5280
Epoch 13/30 | Train Loss: 0.3454 | Val Loss: 0.5299
Epoch 14/30 | Train Loss: 0.3365 | Val Loss: 0.5324
Epoch 15/30 | Train Loss: 0.3282 | Val Loss: 0.5340
Epoch 16/30 | Train Loss: 0.3205 | Val Loss: 0.5376
Epoch 17/30 | Train Loss: 0.3134 | Val Loss: 0.5399
Epoch 18/30 | Train Loss: 0.3069 | Val Loss: 0.5446
Epoch 19/30 | Train Loss: 0.3009 | Val Loss: 0.5480
Epoch 20/30 | Train L

In [17]:
# 2) Attention Seq2Seq 모델 학습
encoder_attn = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_attn = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train_seq2seq(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_attn,
    decoder=decoder_attn,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    print_every=1
)

Epoch 1/30 | Train Loss: 0.7546 | Val Loss: 0.6140
Epoch 2/30 | Train Loss: 0.5764 | Val Loss: 0.5660
Epoch 3/30 | Train Loss: 0.5275 | Val Loss: 0.5407
Epoch 4/30 | Train Loss: 0.4901 | Val Loss: 0.5211
Epoch 5/30 | Train Loss: 0.4587 | Val Loss: 0.5103
Epoch 6/30 | Train Loss: 0.4327 | Val Loss: 0.5028
Epoch 7/30 | Train Loss: 0.4101 | Val Loss: 0.4960
Epoch 8/30 | Train Loss: 0.3906 | Val Loss: 0.4931
Epoch 9/30 | Train Loss: 0.3729 | Val Loss: 0.4903
Epoch 10/30 | Train Loss: 0.3573 | Val Loss: 0.4901
Epoch 11/30 | Train Loss: 0.3429 | Val Loss: 0.4887
Epoch 12/30 | Train Loss: 0.3298 | Val Loss: 0.4899
Epoch 13/30 | Train Loss: 0.3178 | Val Loss: 0.4890
Epoch 14/30 | Train Loss: 0.3068 | Val Loss: 0.4906
Epoch 15/30 | Train Loss: 0.2966 | Val Loss: 0.4917
Epoch 16/30 | Train Loss: 0.2873 | Val Loss: 0.4925
Epoch 17/30 | Train Loss: 0.2783 | Val Loss: 0.4919
Epoch 18/30 | Train Loss: 0.2702 | Val Loss: 0.4947
Epoch 19/30 | Train Loss: 0.2625 | Val Loss: 0.4962
Epoch 20/30 | Train L

In [19]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smooth = SmoothingFunction().method1

# 단일 문장 평가 함수
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence, tokenizer_ko).unsqueeze(0)
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, _ = decoder(encoder_outputs, encoder_hidden)

        # decoder_outputs: [1, MAX_LENGTH, vocab_size]
        _, topi = decoder_outputs.topk(1)  # [1, MAX_LENGTH, 1]
        decoded_ids = topi.squeeze(0).squeeze(-1)  # [MAX_LENGTH]

        decoded_words = []
        for idx in decoded_ids:
            token_id = idx.item()
            if token_id == EOS_token:
                break
            # PAD/SOS 등은 출력에서 제외
            if token_id in [PAD_token, SOS_token]:
                continue
            decoded_words.append(output_lang.index2word.get(token_id, "<unk>"))

    return decoded_words  # 토큰 리스트


# 무작위 문장 쌍 평가 + BLEU 계산
def evaluateRandomly_with_bleu(encoder, decoder, pairs_source, 
                               input_lang, output_lang,
                               tokenizer_ko, tokenizer_en,
                               n=5):
    """
    pairs_source: 평가에 사용할 (ko, en) 쌍 리스트 (ex. pairs_valid)
    n: 이 중 몇 개를 샘플링할지
    """
    # 1) 평가에 사용할 문장 쌍 샘플링 (고정 샘플로 쓰고 싶으면 밖에서 미리 정해도 됨)
    sample_pairs = random.sample(pairs_source, n)

    bleu_scores = []

    for ko, en in sample_pairs:
        print("> KO:", ko)
        print("= EN:", en)

        # 모델 번역
        output_words = evaluate(encoder, decoder, ko, input_lang, output_lang)
        output_sentence = " ".join(output_words)
        print("< MT:", output_sentence)

        # BLEU 계산
        ref_tokens = tokenizer_en(en)
        hyp_tokens = output_words

        score = sentence_bleu(
            [ref_tokens],
            hyp_tokens,
            smoothing_function=smooth
        )
        bleu_scores.append(score)
        print("BLEU:", f"{score:.4f}")
        print()

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU over {n} sentences: {avg_bleu:.4f}")
    return avg_bleu, sample_pairs

In [20]:
print("=== Seq2Seq ===")
seq2seq_bleu, sample_pairs = evaluateRandomly_with_bleu(
    encoder_seq2seq, decoder_seq2seq,
    pairs_valid,      # 평가용 쌍 (검증/테스트 중 택1)
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=5
)

print("=== Seq2Seq + Attention ===")
# 동일 sample_pairs를 사용해 같은 문장에 대해 비교
attn_bleu, _ = evaluateRandomly_with_bleu(
    encoder_attn, decoder_attn,
    sample_pairs,      # 이미 뽑힌 5개 쌍
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=len(sample_pairs)
)

=== Seq2Seq ===
> KO: 기수 지망생으로서, 할 수 있는 한, 분간하기 위해 노력하라.
= EN: As an aspiring rider, try to be as discerning as you can be.
< MT: I 'll make sure to get back to me , and I 've been waiting for the room period .
BLEU: 0.0127

> KO: 또한 잠금 장치가 없는 곳에 돈이나 신용 카드 정보를 남겨두는 것이 조금 꺼림칙합니다.
= EN: I would also feel uncomfortable leaving my money or credit card details in an unlocked environment.
< MT: Also , if you have any other questions , please feel free to contact us at any time .
BLEU: 0.0121

> KO: 친애하는 AAA씨, 귀하가 주문을 취소하기 원하신다는 것을 들었습니다.
= EN: Dear Mr. AAA, I have heard that you want to cancel your order.
< MT: Dear Mr. AAA , I am sending you an email to send a refund of your order number .
BLEU: 0.2383

> KO: 이 정보에는 저희가 지난주 수요일날 귀사로 네 박스를 발송한 것으로 나타나있습니다.
= EN: This information indicates that we shipped four boxes to you last Wednesday.
< MT: We would like to inform you that the exact date of the product will be held at the lowest price .
BLEU: 0.0129

> KO: 건조 시에 옷감이 구겨지는 것을 방지해 줍니다.
= EN:

## 4. 두 모델의 성능 평가 및 개선점 도출  


### 4.1. Seq2Seq vs Attention 번역 품질 정성 평가

- Seq2Seq
    - Seq2Seq는 전체 문장 구조나 정중한 표현 형태는 어느 정도 유지하지만, 핵심 의미가 크게 왜곡되는 사례가 빈번했다.  
        - 예시로, “금지 물품에 대해서는 몰랐습니다.”라는 문장이 “제품을 몰랐다”와 같은 엉뚱한 내용으로 번역되거나,
        - “건조 시에 옷감이 구겨지는 것을 방지해 줍니다.”가 “편리한 소재” 수준의 일반적인 문장으로 희석되는 식이다.
    - 즉, “상황에 맞는 공손한 영문 문장”은 만드는 편이지만, 원문의 구체적인 정보(금지 물품, 구겨짐 방지, 주문 취소 등)를 정확히 반영하는 데는 한계가 있었다.

- Seq2Seq + Attention
    - Attention 모델도 여전히 오역과 의미 손실이 있지만, 일부 문장에서 핵심 정보와 구조를 더 잘 보존하는 모습이 관찰되었다.  
        - 예시로, “친애하는 AAA씨, 귀하가 주문을 취소하기 원하신다는 것을 들었습니다.” 문장에서 Seq2Seq는 환불 메일로 의미가 틀어지지만,  
        - Seq2Seq + Attention은 “~를 원한다는 것을 들었다”는 핵심 구조를 어느 정도 반영한 문장을 생성했다.
    - 다만 다른 예시들에서는 여전히 “다른 상황에서 가져온 듯한 문장”이 생성되는 등, Attention이라고 해서 항상 자연스럽거나 정확한 것은 아니며, 전체적으로는 “조금 더 나은 정도”의 개선으로 볼 수 있다.

- 두 모델 모두 여러 예시에서 문맥과 맞지 않는 표현이나 정보 왜곡이 자주 발생해, 현재 품질은 실무 수준에는 한참 미치지 못하는 기초적인 번역 성능으로 평가된다.

### 4.2. 정량 지표 비교 및 오번역 사례 분석

- 에폭별 손실 곡선 비교

    - Seq2Seq는 Train Loss가 꾸준히 감소하지만, Val Loss는 약 5–10 에폭 구간에서 최저에 도달한 뒤 이후 계속 증가해 명확한 과적합 패턴을 보인다.
    - Attention 모델은 같은 에폭에서 항상 더 낮은 Val Loss를 유지하며, 증가 속도도 더 완만해 일반화 성능이 상대적으로 우수한 것으로 해석할 수 있다.

        | Model              | Epoch | Train Loss | Val Loss |
        |--------------------|:-----:|-----------:|---------:|
        | Seq2Seq            |   5   | 0.4655     | 0.5250   |
        | Seq2Seq            |  10   | 0.3783     | 0.5225   |
        | Seq2Seq            |  15   | 0.3282     | 0.5340   |
        | Seq2Seq            |  20   | 0.2952     | 0.5481   |
        | Seq2Seq            |  25   | 0.2716     | 0.5642   |
        | Seq2Seq            |  30   | 0.2538     | 0.5788   |
        | Seq2Seq + Attention|   5   | 0.4587     | 0.5103   |
        | Seq2Seq + Attention|  10   | 0.3573     | 0.4901   |
        | Seq2Seq + Attention|  15   | 0.2966     | 0.4917   |
        | Seq2Seq + Attention|  20   | 0.2554     | 0.4977   |
        | Seq2Seq + Attention|  25   | 0.2255     | 0.5077   |
        | Seq2Seq + Attention|  30   | 0.2021     | 0.5136   |

- BLEU 기반 정량 비교
    - 동일한 5개 검증 문장을 샘플로 선택해, 각 문장에 대한 BLEU를 계산하고 평균을 비교한 결과는 다음과 같다.  
        - Seq2Seq: 평균 BLEU ≈ 0.0586  
        - Seq2Seq + Attention: 평균 BLEU ≈ 0.0822
    - 절대 점수는 매우 낮지만, 동일 조건에서 Attention 모델이 Seq2Seq보다 일관되게 더 높은 BLEU를 기록해, 정성 평가에서 관찰된 “의미 보존 측면에서의 소폭 우위”를 수치적으로도 확인할 수 있다.

- 오번역 패턴 분석
    - Seq2Seq
        - 전체 문장 톤은 공손한 비즈니스 영어에 가깝지만, 핵심 정보(금지 물품, 옷감 구겨짐 방지, 배송 박스 수량 등)가 아예 다른 내용으로 바뀌는 경우가 많았다.
        - “건조 시에 옷감이 구겨지는 것을 방지해 줍니다.” → “편리한 소재” 정도의 일반 문장으로 흐려지는 식으로, 의미가 구체성·정확성을 잃는 경향이 있다.

    - Seq2Seq + Attention
        - 일부 문장에서 주문 취소, 이메일 수신 등 핵심 사건 구조를 비교적 잘 유지했지만, 다른 문장들에서는 여전히 엉뚱한 상황 설명이 등장하는 등 오번역이 잦았다.
        - 예를 들어, “친애하는 AAA씨, 귀하가 주문을 취소하기 원하신다는 것을 들었습니다.” 문장은 Seq2Seq보다 원문의 의도를 더 잘 반영했지만, 다른 예시에서는 숫자·수량·상세 조건이 왜곡되거나 문맥이 맞지 않는 표현이 생성되었다.

### 개선점 정리

- 모델 구조·하이퍼파라미터 튜닝
    - Attention 모델은 이미 Seq2Seq보다 나은 일반화 성능을 보이고 있으므로, 이 구조를 기준으로 hidden size(예: 128→256)나 레이어 수를 조정해 용량을 늘리는 실험을 할 수 있다.
    - 학습률(예: 0.001 vs 0.0005)과 배치 크기 변화도 후보이며, 각 실험에서는 한 번에 한 가지 요소만 변경해 어떤 설정이 Train/Val Loss와 BLEU에 긍정적인 영향을 주었는지 명확히 해석할 수 있게 하는 것이 중요하다.

- 학습 설정
    - 두 모델 모두 약 15~20 에폭을 기점으로 Val Loss가 뚜렷하게 상승하므로, 이 구간을 Early Stopping 후보로 두는 것이 합리적이다.
    - Seq2Seq는 Val Loss 바닥이 5~10 에폭 사이에 위치해 더 짧은 학습(예: 10~15 에폭)으로도 과적합을 줄일 수 있고, Attention 모델은 15~20 에폭 구간에서 충분히 학습된 뒤 Val Loss가 서서히 증가하는 패턴이므로 약간 더 긴 학습 시간(예: 최대 20 에폭)까지 허용할 수 있다.

- 데이터 전처리
    - 현재 문장에는 숫자, 단위, 괄호·기호(예: “(주)”, “-”, “%”) 등이 다양한 형태로 섞여 있어, 번역 모델 입장에서 불필요한 변형이나 희귀 토큰을 많이 만든다.
    - 숫자·단위를 일정한 포맷으로 정규화하고, 불필요한 특수문자·중복 공백을 정리하는 전처리 실험을 통해, 노이즈를 줄이고 핵심 단어 분포를 더 안정적으로 만들 여지가 있다.

- 종합 개선 방향
    1) Attention 모델 중심의 hidden size·학습률  
    2) Early Stopping 도입 및 에폭 단축
    3) 전처리 개선
    
    위 실험을 단계적으로 진행해 성능 향상을 시도함

## 5. 추가 실험


### 5.1. 하이퍼파라미터 및 모델 구조 변경 실험  
**[추가 실험 1]**

- 실험 아이디어
    - Attention 모델을 기준으로 hidden size를 128→256으로 키워 표현력을 높이고, 같은 조건에서 성능 변화를 본다.

- 관찰 포인트
    - 더 큰 hidden size에서 Val Loss와 BLEU가 실제로 개선되는지, 아니면 과적합만 빨라지는지 확인

**[추가실험 1] hidden size 변경**
- hidden_size=256으로 인코더/디코더 초기화(나머지 설정 동일)

In [ ]:
# 추가실험 1: hidden size 256 (Seq2Seq, Attention 모두)

hidden_size_large = 256
n_epochs = 30          # 또는 Early Stopping 포함 최대 에폭
learning_rate = 0.001  # baseline과 동일

In [16]:
# 1) Seq2Seq (hidden=256)
encoder_seq2seq_256 = EncoderRNN(input_lang.n_words, hidden_size_large).to(device)
decoder_seq2seq_256 = DecoderRNN(hidden_size_large, output_lang.n_words).to(device)

train_seq2seq(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_seq2seq_256,
    decoder=decoder_seq2seq_256,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    print_every=1
)

Epoch 1/30 | Train Loss: 0.6787 | Val Loss: 0.5681
Epoch 2/30 | Train Loss: 0.5200 | Val Loss: 0.5251
Epoch 3/30 | Train Loss: 0.4621 | Val Loss: 0.5070
Epoch 4/30 | Train Loss: 0.4173 | Val Loss: 0.5009
Epoch 5/30 | Train Loss: 0.3799 | Val Loss: 0.4996
Epoch 6/30 | Train Loss: 0.3485 | Val Loss: 0.5050
Epoch 7/30 | Train Loss: 0.3220 | Val Loss: 0.5099
Epoch 8/30 | Train Loss: 0.2996 | Val Loss: 0.5153
Epoch 9/30 | Train Loss: 0.2805 | Val Loss: 0.5219
Epoch 10/30 | Train Loss: 0.2642 | Val Loss: 0.5290
Epoch 11/30 | Train Loss: 0.2498 | Val Loss: 0.5375
Epoch 12/30 | Train Loss: 0.2368 | Val Loss: 0.5445
Epoch 13/30 | Train Loss: 0.2252 | Val Loss: 0.5537
Epoch 14/30 | Train Loss: 0.2149 | Val Loss: 0.5612
Epoch 15/30 | Train Loss: 0.2057 | Val Loss: 0.5697
Epoch 16/30 | Train Loss: 0.1973 | Val Loss: 0.5774
Epoch 17/30 | Train Loss: 0.1897 | Val Loss: 0.5853
Epoch 18/30 | Train Loss: 0.1828 | Val Loss: 0.5944
Epoch 19/30 | Train Loss: 0.1764 | Val Loss: 0.6034
Epoch 20/30 | Train L

In [17]:
print("=== Seq2Seq (hidden=256) ===")
seq2seq_bleu_256, sample_pairs_h256 = evaluateRandomly_with_bleu(
    encoder_seq2seq_256, decoder_seq2seq_256,
    pairs_valid,
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=5
)

=== Seq2Seq (hidden=256) ===
> KO: 아내분이 소녀때의 꿈을 떠올리시며 좋아하실 것 같아요.
= EN: I think your wife will like it, reminding her of her childhood dreams.
< MT: When I was young , I was actually tired of my mind , so I realized it to me .
BLEU: 0.0144

> KO: >자, 이렇게 리허설하는 거예요.
= EN: We're doing rehearsals like this.
< MT: > Like this , this is the best .
BLEU: 0.0285

> KO: 네, 미국에서 정비사가 파견가는 것보다 빠르게 정비가 가능할 것입니다.
= EN: Yes, maintenance will be possible faster than a mechanic dispatched from the United States.
< MT: Yes , it is possible to release foreign markets , so the Korean government will be upgraded right in the market cover in the car .
BLEU: 0.0250

> KO: 귀하께서 이번에 진행하시는 지질학 연구에 대한 협조를 요청하셨지요.
= EN: You have requested your cooperation in this geological research.
< MT: You asked us to send you our transaction with our company .
BLEU: 0.0203

> KO: 일이 마무리되는 대로 다시 프레젠테이션 일정을 잡을 예정입니다.
= EN: We will reschedule the presentation as soon as work is finished.
< MT: If you need to change the meetin

In [18]:
# 2) Seq2Seq + Attention (hidden=256)
encoder_attn_256 = EncoderRNN(input_lang.n_words, hidden_size_large).to(device)
decoder_attn_256 = AttnDecoderRNN(hidden_size_large, output_lang.n_words).to(device)

train_seq2seq(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_attn_256,
    decoder=decoder_attn_256,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    print_every=1
)

Epoch 1/30 | Train Loss: 0.6651 | Val Loss: 0.5620
Epoch 2/30 | Train Loss: 0.5117 | Val Loss: 0.5089
Epoch 3/30 | Train Loss: 0.4438 | Val Loss: 0.4845
Epoch 4/30 | Train Loss: 0.3903 | Val Loss: 0.4729
Epoch 5/30 | Train Loss: 0.3467 | Val Loss: 0.4694
Epoch 6/30 | Train Loss: 0.3105 | Val Loss: 0.4670
Epoch 7/30 | Train Loss: 0.2804 | Val Loss: 0.4698
Epoch 8/30 | Train Loss: 0.2559 | Val Loss: 0.4728
Epoch 9/30 | Train Loss: 0.2357 | Val Loss: 0.4776
Epoch 10/30 | Train Loss: 0.2189 | Val Loss: 0.4825
Epoch 11/30 | Train Loss: 0.2046 | Val Loss: 0.4885
Epoch 12/30 | Train Loss: 0.1924 | Val Loss: 0.4956
Epoch 13/30 | Train Loss: 0.1818 | Val Loss: 0.5012
Epoch 14/30 | Train Loss: 0.1726 | Val Loss: 0.5067
Epoch 15/30 | Train Loss: 0.1645 | Val Loss: 0.5125
Epoch 16/30 | Train Loss: 0.1569 | Val Loss: 0.5194
Epoch 17/30 | Train Loss: 0.1506 | Val Loss: 0.5256
Epoch 18/30 | Train Loss: 0.1446 | Val Loss: 0.5333
Epoch 19/30 | Train Loss: 0.1392 | Val Loss: 0.5395
Epoch 20/30 | Train L

In [19]:
print("=== Seq2Seq + Attention (hidden=256) ===")
attn_bleu_256, _ = evaluateRandomly_with_bleu(
    encoder_attn_256, decoder_attn_256,
    sample_pairs_h256,   # 같은 문장 5개로 비교
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=len(sample_pairs_h256)
)

=== Seq2Seq + Attention (hidden=256) ===
> KO: 아내분이 소녀때의 꿈을 떠올리시며 좋아하실 것 같아요.
= EN: I think your wife will like it, reminding her of her childhood dreams.
< MT: I think your hair will also lead my fishing dad .
BLEU: 0.0737

> KO: >자, 이렇게 리허설하는 거예요.
= EN: We're doing rehearsals like this.
< MT: > Now , it 's this one .
BLEU: 0.0330

> KO: 귀하께서 이번에 진행하시는 지질학 연구에 대한 협조를 요청하셨지요.
= EN: You have requested your cooperation in this geological research.
< MT: I have a request for the cooperation design this time .
BLEU: 0.0267

> KO: 네, 미국에서 정비사가 파견가는 것보다 빠르게 정비가 가능할 것입니다.
= EN: Yes, maintenance will be possible faster than a mechanic dispatched from the United States.
< MT: Yes , the United States will be in the U.S. , and there will be able to access more than in communication .
BLEU: 0.0642

> KO: 일이 마무리되는 대로 다시 프레젠테이션 일정을 잡을 예정입니다.
= EN: We will reschedule the presentation as soon as work is finished.
< MT: We will be conducting brainstorming schedule presentation .
BLEU: 0.0424

Average B

**[추가 실험 2]**

- 실험 아이디어
    - 학습률(예: 0.001 vs 0.0005)만 단독으로 바꿔, 수렴 속도와 과적합 패턴이 어떻게 달라지는지 비교한다.

- 관찰 포인트
    - 학습률을 낮췄을 때 Train Loss 감소 속도는 느려지지만 Val Loss 곡선이 더 안정적으로 변하는지 확인

**[추가실험 2] learing late 변경**
- learning_rate=0.0005 으로 초기화(나머지 설정 동일)

In [ ]:
# 추가실험 2: lr = 0.0005 (Seq2Seq, Attention 모두)

hidden_size = 128       # 구조는 baseline과 동일
n_epochs = 30           # 또는 Early Stopping 포함 최대 에폭
learning_rate_small = 0.0005

In [21]:
# 1) Seq2Seq (lr=0.0005)
encoder_seq2seq_lr2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_seq2seq_lr2 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train_seq2seq(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_seq2seq_lr2,
    decoder=decoder_seq2seq_lr2,
    n_epochs=n_epochs,
    learning_rate=learning_rate_small,
    print_every=1
)

Epoch 1/30 | Train Loss: 0.8897 | Val Loss: 0.6777
Epoch 2/30 | Train Loss: 0.6391 | Val Loss: 0.6268
Epoch 3/30 | Train Loss: 0.5895 | Val Loss: 0.5903
Epoch 4/30 | Train Loss: 0.5537 | Val Loss: 0.5703
Epoch 5/30 | Train Loss: 0.5277 | Val Loss: 0.5569
Epoch 6/30 | Train Loss: 0.5070 | Val Loss: 0.5478
Epoch 7/30 | Train Loss: 0.4896 | Val Loss: 0.5410
Epoch 8/30 | Train Loss: 0.4746 | Val Loss: 0.5374
Epoch 9/30 | Train Loss: 0.4614 | Val Loss: 0.5334
Epoch 10/30 | Train Loss: 0.4496 | Val Loss: 0.5318
Epoch 11/30 | Train Loss: 0.4385 | Val Loss: 0.5315
Epoch 12/30 | Train Loss: 0.4286 | Val Loss: 0.5290
Epoch 13/30 | Train Loss: 0.4191 | Val Loss: 0.5289
Epoch 14/30 | Train Loss: 0.4104 | Val Loss: 0.5292
Epoch 15/30 | Train Loss: 0.4023 | Val Loss: 0.5296
Epoch 16/30 | Train Loss: 0.3945 | Val Loss: 0.5290
Epoch 17/30 | Train Loss: 0.3873 | Val Loss: 0.5305
Epoch 18/30 | Train Loss: 0.3804 | Val Loss: 0.5304
Epoch 19/30 | Train Loss: 0.3741 | Val Loss: 0.5320
Epoch 20/30 | Train L

In [22]:
print("=== Seq2Seq (lr=0.0005) ===")
seq2seq_bleu_lr2, sample_pairs_lr2 = evaluateRandomly_with_bleu(
    encoder_seq2seq_lr2, decoder_seq2seq_lr2,
    pairs_valid,
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=5
)

=== Seq2Seq (lr=0.0005) ===
> KO: >사막에 물고기 있는 거 알아?
= EN: Do you know there's fish in the desert?
< MT: > I think it 's a little bit of this .
BLEU: 0.0189

> KO: 그라인더에 모래가 부족할 때 어디서 구하나요?
= EN: Where can I get it when my grinder is running out of sand?
< MT: How much is the wholesale price for sanitizing ?
BLEU: 0.0164

> KO: 그래서 4월 19일에 귀하께 최신 카달로그를 요청하는 메일을 보내 드렸습니다.
= EN: So, on April 19th, we sent you an email requesting the latest catalog.
< MT: So we will send you a detailed profile of the contract with you to send the latest catalog .
BLEU: 0.1458

> KO: 당신의 이중 디스플레이를 디스플레이 포트 디스플레이에 연결하세요.
= EN: Connect your dual display to a DisplayPort display.
< MT: It is a lightweight and foam cleanser .
BLEU: 0.0292

> KO: 잘 쓰고 있습니다.
= EN: I'm using it well.
< MT: You have to do it .
BLEU: 0.0485

Average BLEU over 5 sentences: 0.0517


In [23]:
# 2) Seq2Seq + Attention (lr=0.0005)
encoder_attn_lr2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_attn_lr2 = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train_seq2seq(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_attn_lr2,
    decoder=decoder_attn_lr2,
    n_epochs=n_epochs,
    learning_rate=learning_rate_small,
    print_every=1
)

Epoch 1/30 | Train Loss: 0.8519 | Val Loss: 0.6570
Epoch 2/30 | Train Loss: 0.6167 | Val Loss: 0.5998
Epoch 3/30 | Train Loss: 0.5698 | Val Loss: 0.5700
Epoch 4/30 | Train Loss: 0.5369 | Val Loss: 0.5508
Epoch 5/30 | Train Loss: 0.5114 | Val Loss: 0.5373
Epoch 6/30 | Train Loss: 0.4901 | Val Loss: 0.5272
Epoch 7/30 | Train Loss: 0.4718 | Val Loss: 0.5197
Epoch 8/30 | Train Loss: 0.4549 | Val Loss: 0.5126
Epoch 9/30 | Train Loss: 0.4400 | Val Loss: 0.5071
Epoch 10/30 | Train Loss: 0.4261 | Val Loss: 0.5027
Epoch 11/30 | Train Loss: 0.4133 | Val Loss: 0.4993
Epoch 12/30 | Train Loss: 0.4015 | Val Loss: 0.4966
Epoch 13/30 | Train Loss: 0.3905 | Val Loss: 0.4942
Epoch 14/30 | Train Loss: 0.3799 | Val Loss: 0.4923
Epoch 15/30 | Train Loss: 0.3701 | Val Loss: 0.4904
Epoch 16/30 | Train Loss: 0.3608 | Val Loss: 0.4893
Epoch 17/30 | Train Loss: 0.3519 | Val Loss: 0.4877
Epoch 18/30 | Train Loss: 0.3435 | Val Loss: 0.4874
Epoch 19/30 | Train Loss: 0.3354 | Val Loss: 0.4873
Epoch 20/30 | Train L

In [24]:
print("=== Seq2Seq + Attention (lr=0.0005) ===")
attn_bleu_lr2, _ = evaluateRandomly_with_bleu(
    encoder_attn_lr2, decoder_attn_lr2,
    sample_pairs_lr2,     # 같은 문장 5개로 비교
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=len(sample_pairs_lr2)
)

=== Seq2Seq + Attention (lr=0.0005) ===
> KO: 당신의 이중 디스플레이를 디스플레이 포트 디스플레이에 연결하세요.
= EN: Connect your dual display to a DisplayPort display.
< MT: Your eyes have a strong image skin .
BLEU: 0.0292

> KO: 잘 쓰고 있습니다.
= EN: I'm using it well.
< MT: You have to see well .
BLEU: 0.0863

> KO: >사막에 물고기 있는 거 알아?
= EN: Do you know there's fish in the desert?
< MT: > I know that there are many people in the middle .
BLEU: 0.0428

> KO: 그래서 4월 19일에 귀하께 최신 카달로그를 요청하는 메일을 보내 드렸습니다.
= EN: So, on April 19th, we sent you an email requesting the latest catalog.
< MT: So I have sending you an email to request a catalog of the latest edition ceremony .
BLEU: 0.0805

> KO: 그라인더에 모래가 부족할 때 어디서 구하나요?
= EN: Where can I get it when my grinder is running out of sand?
< MT: What is the difference in the middle of the US ?
BLEU: 0.0189

Average BLEU over 5 sentences: 0.0515


### 5.2. 학습 설정 변경 실험

**[추가실험 3]**

- 실험 아이디어
    - 기존 베이스라인과 동일한 데이터, 모델 구조, 하이퍼파라미터를 사용하되, Early Stopping을 적용한 학습 루프를 도입해 Seq2Seq와 Seq2Seq+Attention 모두에서 과적합 구간 이전에 학습을 자동으로 중단해 본다.
    - 검증 손실(Val Loss)이 일정 횟수 연속으로 개선되지 않으면 학습을 멈추는 방식으로, 불필요한 에폭을 줄이면서 Val Loss와 BLEU가 유지 또는 개선되는지 확인한다.

- 기대 효과 및 관찰 포인트
    - 에폭 수가 줄어들어 총 학습 시간이 단축되면서도, Val Loss 최소값과 BLEU가 베이스라인 대비 유지되거나 약간이라도 개선되는지 확인한다.
    - Seq2Seq와 Attention 모두에서 Early Stopping이 적용되는 에폭 지점을 비교해, 어떤 모델이 더 빨리 과적합 구간에 진입하는지도 함께 관찰한다.

In [ ]:
# 추가실험 3: Early Stopping용 학습 루프 (기본 train_epoch, evaluate_epoch 그대로 사용)

def train_seq2seq_with_early_stopping(
    train_dataloader,
    valid_dataloader,
    encoder,
    decoder,
    n_epochs,
    learning_rate=0.001,
    patience=3,              # 개선이 없을 때 허용할 에폭 수
    print_every=1
):
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    best_val_loss = float("inf")
    best_state = None
    epochs_no_improve = 0

    for epoch in range(1, n_epochs + 1):
        train_loss = train_epoch(
            train_dataloader, encoder, decoder,
            encoder_optimizer, decoder_optimizer, criterion
        )
        val_loss = evaluate_epoch(
            valid_dataloader, encoder, decoder, criterion
        )

        # 개선 여부 체크
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            # 가장 좋은 상태 저장
            best_state = {
                "encoder": encoder.state_dict(),
                "decoder": decoder.state_dict()
            }
        else:
            epochs_no_improve += 1

        if epoch % print_every == 0:
            print(
                f"[EarlyStop] Epoch {epoch}/{n_epochs} "
                f"| Train Loss: {train_loss:.4f} "
                f"| Val Loss: {val_loss:.4f} "
                f"| NoImprove: {epochs_no_improve}/{patience}"
            )

        # patience 만큼 개선이 없으면 학습 조기 종료
        if epochs_no_improve >= patience:
            print(f"> Early stopping triggered at epoch {epoch}")
            break

    # 가장 좋은 가중치로 되돌리기
    if best_state is not None:
        encoder.load_state_dict(best_state["encoder"])
        decoder.load_state_dict(best_state["decoder"])

    return best_val_loss

In [15]:
hidden_size = 128
n_epochs = 30          # 최대 에폭 (실제로는 더 일찍 멈출 수 있음)
learning_rate = 0.001
patience = 3           # Val Loss 3에폭 연속 악화 시 중단

In [16]:
encoder_seq2seq_es = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_seq2seq_es = DecoderRNN(hidden_size, output_lang.n_words).to(device)

print("\n=== [추가실험 3-1] Seq2Seq + Early Stopping ===")
best_val_loss_seq2seq_es = train_seq2seq_with_early_stopping(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_seq2seq_es,
    decoder=decoder_seq2seq_es,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    patience=patience,
    print_every=1
)
print(f"Best Val Loss (Seq2Seq + ES): {best_val_loss_seq2seq_es:.4f}")


=== [추가실험 3-1] Seq2Seq + Early Stopping ===
[EarlyStop] Epoch 1/30 | Train Loss: 0.7914 | Val Loss: 0.6386 | NoImprove: 0/3
[EarlyStop] Epoch 2/30 | Train Loss: 0.5907 | Val Loss: 0.5843 | NoImprove: 0/3
[EarlyStop] Epoch 3/30 | Train Loss: 0.5399 | Val Loss: 0.5594 | NoImprove: 0/3
[EarlyStop] Epoch 4/30 | Train Loss: 0.5053 | Val Loss: 0.5452 | NoImprove: 0/3
[EarlyStop] Epoch 5/30 | Train Loss: 0.4788 | Val Loss: 0.5381 | NoImprove: 0/3
[EarlyStop] Epoch 6/30 | Train Loss: 0.4562 | Val Loss: 0.5340 | NoImprove: 0/3
[EarlyStop] Epoch 7/30 | Train Loss: 0.4367 | Val Loss: 0.5317 | NoImprove: 0/3
[EarlyStop] Epoch 8/30 | Train Loss: 0.4196 | Val Loss: 0.5322 | NoImprove: 1/3
[EarlyStop] Epoch 9/30 | Train Loss: 0.4047 | Val Loss: 0.5325 | NoImprove: 2/3
[EarlyStop] Epoch 10/30 | Train Loss: 0.3914 | Val Loss: 0.5349 | NoImprove: 3/3
> Early stopping triggered at epoch 10
Best Val Loss (Seq2Seq + ES): 0.5317


In [20]:
print("\n=== [추가실험 3-1] Seq2Seq + Early Stopping: BLEU 평가 ===")
seq2seq_es_bleu, sample_pairs_es = evaluateRandomly_with_bleu(
    encoder_seq2seq_es, decoder_seq2seq_es,
    pairs_valid,          # 기존과 동일한 검증 쌍 리스트
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=5                   # 샘플 개수
)


=== [추가실험 3-1] Seq2Seq + Early Stopping: BLEU 평가 ===
> KO: 예, 선택할 수 있는 항목이 많습니다.
= EN: Yes, we have a lot of those in our selection.
< MT: Yes , it is a good choice to be a good time .
BLEU: 0.0391

> KO: > 내 삶의 반이라도 알아봐 준다면 나는 정말 좋겠네
= EN: > If you recognize half of my life, I'd really like it.
< MT: > I 'm not going to eat it like this .
BLEU: 0.0196

> KO: >그래요.
= EN: >Yes.
< MT: > Yes .
BLEU: 0.5623

> KO: 일별 근무기록을 통해 시급, 일급, 시급+일급 등 다양한 형태의 급여 계산방식을 사용할 수 있습니다.
= EN: Through daily work records, you can use various types of salary calculation methods such as hourly wage, daily wage, hourly wage + daily wage.
< MT: The headquarters can be used for a long time , and the quality of the main ingredient and the quantity of the voice recognition is to be a huge loss for the first time .
BLEU: 0.0075

> KO: 더 많은 단서가 필요하다고 생각하시면 저희가 제공하겠습니다.
= EN: If you think that you need more clues, we will provide them.
< MT: We hope that you will be able to receive the product .
BLEU: 0.0363

Average

In [17]:
encoder_attn_es = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_attn_es = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

print("\n=== [추가실험 3-2] Seq2Seq + Attention + Early Stopping ===")
best_val_loss_attn_es = train_seq2seq_with_early_stopping(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    encoder=encoder_attn_es,
    decoder=decoder_attn_es,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    patience=patience,
    print_every=1
)
print(f"Best Val Loss (Attn + ES): {best_val_loss_attn_es:.4f}")


=== [추가실험 3-2] Seq2Seq + Attention + Early Stopping ===
[EarlyStop] Epoch 1/30 | Train Loss: 0.7534 | Val Loss: 0.6113 | NoImprove: 0/3
[EarlyStop] Epoch 2/30 | Train Loss: 0.5709 | Val Loss: 0.5599 | NoImprove: 0/3
[EarlyStop] Epoch 3/30 | Train Loss: 0.5191 | Val Loss: 0.5308 | NoImprove: 0/3
[EarlyStop] Epoch 4/30 | Train Loss: 0.4804 | Val Loss: 0.5141 | NoImprove: 0/3
[EarlyStop] Epoch 5/30 | Train Loss: 0.4497 | Val Loss: 0.5039 | NoImprove: 0/3
[EarlyStop] Epoch 6/30 | Train Loss: 0.4239 | Val Loss: 0.4973 | NoImprove: 0/3
[EarlyStop] Epoch 7/30 | Train Loss: 0.4015 | Val Loss: 0.4924 | NoImprove: 0/3
[EarlyStop] Epoch 8/30 | Train Loss: 0.3819 | Val Loss: 0.4895 | NoImprove: 0/3
[EarlyStop] Epoch 9/30 | Train Loss: 0.3646 | Val Loss: 0.4884 | NoImprove: 0/3
[EarlyStop] Epoch 10/30 | Train Loss: 0.3487 | Val Loss: 0.4876 | NoImprove: 0/3
[EarlyStop] Epoch 11/30 | Train Loss: 0.3342 | Val Loss: 0.4872 | NoImprove: 0/3
[EarlyStop] Epoch 12/30 | Train Loss: 0.3212 | Val Loss: 0.48

In [21]:
print("\n=== [추가실험 3-2] Attn + Early Stopping: BLEU 평가 ===")
# 동일 sample_pairs_es를 사용해 같은 문장에 대해 비교
attn_es_bleu, _ = evaluateRandomly_with_bleu(
    encoder_attn_es, decoder_attn_es,
    sample_pairs_es,      # 위에서 뽑은 동일 5개 쌍
    input_lang, output_lang,
    tokenizer_ko, tokenizer_en,
    n=len(sample_pairs_es)
)


=== [추가실험 3-2] Attn + Early Stopping: BLEU 평가 ===
> KO: 일별 근무기록을 통해 시급, 일급, 시급+일급 등 다양한 형태의 급여 계산방식을 사용할 수 있습니다.
= EN: Through daily work records, you can use various types of salary calculation methods such as hourly wage, daily wage, hourly wage + daily wage.
< MT: Through the Freshjeon trimmer , we can use the unit price of the other companies of the other companies of the remote control of the remote control of the remote control .
BLEU: 0.0162

> KO: 예, 선택할 수 있는 항목이 많습니다.
= EN: Yes, we have a lot of those in our selection.
< MT: Yes , there are many models that can be a foreigner .
BLEU: 0.0428

> KO: 더 많은 단서가 필요하다고 생각하시면 저희가 제공하겠습니다.
= EN: If you think that you need more clues, we will provide them.
< MT: We will prepare for the more but we will prepare .
BLEU: 0.0361

> KO: >그래요.
= EN: >Yes.
< MT: > Yes .
BLEU: 0.5623

> KO: > 내 삶의 반이라도 알아봐 준다면 나는 정말 좋겠네
= EN: > If you recognize half of my life, I'd really like it.
< MT: > I 'm Kim Lee Kwangsoo , and I was really nervous .
BLEU

### 5.3. 전처리 방법 변경(정규화) 실험

**[추가 실험 4]**

- 실험 아이디어
    - 숫자(날짜·수량)나 특수문자(괄호, 기호) 정규화를 추가하여, 번역이 어려운 노이즈를 줄이는 방식을 시도한다.

- 기대 효과 및 관찰 포인트
    - Val Loss 변화: 과적합 시점이 늦춰지는지, 전체 손실 수준이 낮아지는지 확인
    - BLEU 변화: 짧은 문장, 긴 문장에서 각각 의미 보존이 나아지는지 확인

**[추가실험 4] 전처리 변경**

- MAX_LENGTH 설정을 유지하되, 숫자·단위·특수문자 정규화 등 텍스트 전처리만 추가 적용한 후, 같은 에폭·학습률 조건에서 두 모델을 다시 학습시켜 Train/Val Loss와 BLEU를 비교한다.
​
- 문장 내용은 그대로 두고 표기 방식만 정리함으로써, 너무 긴 문장을 자르지 않고도 노이즈를 줄여, 의미 보존과 학습 안정성이 개선되는지를 확인하는 실험이다.

In [22]:
# 추가실험 4: 전처리 함수 정의
import re

def preprocess_ko(text):
    text = text.strip()
    # 예시: 연속 공백 하나로 통일
    text = re.sub(r"\s+", " ", text)
    # 예시: 괄호 안 내용 제거 (너무 구체적이면 빼도 됨)
    # text = re.sub(r"\([^)]*\)", "", text)
    return text

def preprocess_en(text):
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [23]:
# 베이스라인에서 이미 정의된 ko_sentences_train, mt_sentences_train, ko_sentences_valid, mt_sentences_valid 사용

ko_train_prep = [preprocess_ko(s) for s in ko_sentences_train]
en_train_prep = [preprocess_en(s) for s in mt_sentences_train]

ko_valid_prep = [preprocess_ko(s) for s in ko_sentences_valid]
en_valid_prep = [preprocess_en(s) for s in mt_sentences_valid]

print("Train(preprocessed):", len(ko_train_prep))
print("Valid(preprocessed):", len(ko_valid_prep))
print("Sample(preprocessed KO):", ko_train_prep[0])
print("Sample(preprocessed EN):", en_train_prep[0])

Train(preprocessed): 50000
Valid(preprocessed): 1000
Sample(preprocessed KO): 원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.
Sample(preprocessed EN): If you reply to the color you want, we will start making it right away.


In [24]:
# 전처리 버전 Lang 생성 (기존 Lang 클래스 재사용, 새 인스턴스)
def prepareData_preprocessed(ko_list, en_list, tokenizer1, tokenizer2):
    input_lang_prep = Lang("ko_prep")
    output_lang_prep = Lang("en_prep")
    pairs_prep = list(zip(ko_list, en_list))
    print("Read %s preprocessed sentence pairs" % len(pairs_prep))
    for ko, en in pairs_prep:
        input_lang_prep.addSentence(ko, tokenizer1)
        output_lang_prep.addSentence(en, tokenizer2)
    return input_lang_prep, output_lang_prep, pairs_prep

input_lang_prep, output_lang_prep, pairs_prep = prepareData_preprocessed(
    ko_train_prep, en_train_prep, tokenizer_ko, tokenizer_en
)

pairs_valid_prep = list(zip(ko_valid_prep, en_valid_prep))
print("Read %s preprocessed validation sentence pairs" % len(pairs_valid_prep))
print("Vocab sizes (PREP):", input_lang_prep.n_words, output_lang_prep.n_words)

Read 50000 preprocessed sentence pairs
Read 1000 preprocessed validation sentence pairs
Vocab sizes (PREP): 32474 21703


In [25]:
def get_dataloader_preprocessed(batch_size):
    input_tensors = [
        tensorFromSentence(input_lang_prep, ko, tokenizer_ko) for ko, _ in pairs_prep
    ]
    target_tensors = [
        tensorFromSentence(output_lang_prep, en, tokenizer_en) for _, en in pairs_prep
    ]

    input_tensors = torch.stack(input_tensors, dim=0)
    target_tensors = torch.stack(target_tensors, dim=0)

    dataset = TensorDataset(input_tensors, target_tensors)
    train_sampler = RandomSampler(dataset)
    train_dataloader_prep = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)

    print(f"[PREP] input_tensors.shape: {input_tensors.shape}, target_tensors.shape: {target_tensors.shape}")
    return train_dataloader_prep

def get_valid_dataloader_preprocessed(batch_size):
    input_tensors = [
        tensorFromSentence(input_lang_prep, ko, tokenizer_ko) for ko, _ in pairs_valid_prep
    ]
    target_tensors = [
        tensorFromSentence(output_lang_prep, en, tokenizer_en) for _, en in pairs_valid_prep
    ]

    input_tensors = torch.stack(input_tensors, dim=0)
    target_tensors = torch.stack(target_tensors, dim=0)

    dataset = TensorDataset(input_tensors, target_tensors)
    valid_dataloader_prep = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    print(f"[PREP-VALID] input_tensors.shape: {input_tensors.shape}, target_tensors.shape: {target_tensors.shape}")
    return valid_dataloader_prep

train_dataloader_prep = get_dataloader_preprocessed(batch_size=32)
valid_dataloader_prep = get_valid_dataloader_preprocessed(batch_size=32)

[PREP] input_tensors.shape: torch.Size([50000, 96]), target_tensors.shape: torch.Size([50000, 96])
[PREP-VALID] input_tensors.shape: torch.Size([1000, 96]), target_tensors.shape: torch.Size([1000, 96])


In [26]:
hidden_size = 128
n_epochs = 30
learning_rate = 0.001

In [27]:
# 1) 전처리 버전 Seq2Seq
encoder_seq2seq_prep = EncoderRNN(input_lang_prep.n_words, hidden_size).to(device)
decoder_seq2seq_prep = DecoderRNN(hidden_size, output_lang_prep.n_words).to(device)

print("\n=== [추가실험 4-1] Seq2Seq (전처리만 변경, MAX_LENGTH 유지) ===")
train_seq2seq(
    train_dataloader=train_dataloader_prep,
    valid_dataloader=valid_dataloader_prep,
    encoder=encoder_seq2seq_prep,
    decoder=decoder_seq2seq_prep,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    print_every=1
)


=== [추가실험 4-1] Seq2Seq (전처리만 변경, MAX_LENGTH 유지) ===
Epoch 1/30 | Train Loss: 0.7676 | Val Loss: 0.6203
Epoch 2/30 | Train Loss: 0.5805 | Val Loss: 0.5762
Epoch 3/30 | Train Loss: 0.5344 | Val Loss: 0.5528
Epoch 4/30 | Train Loss: 0.5001 | Val Loss: 0.5387
Epoch 5/30 | Train Loss: 0.4732 | Val Loss: 0.5308
Epoch 6/30 | Train Loss: 0.4513 | Val Loss: 0.5285
Epoch 7/30 | Train Loss: 0.4327 | Val Loss: 0.5288
Epoch 8/30 | Train Loss: 0.4166 | Val Loss: 0.5293
Epoch 9/30 | Train Loss: 0.4022 | Val Loss: 0.5306
Epoch 10/30 | Train Loss: 0.3895 | Val Loss: 0.5347
Epoch 11/30 | Train Loss: 0.3781 | Val Loss: 0.5371
Epoch 12/30 | Train Loss: 0.3677 | Val Loss: 0.5397
Epoch 13/30 | Train Loss: 0.3584 | Val Loss: 0.5414
Epoch 14/30 | Train Loss: 0.3498 | Val Loss: 0.5447
Epoch 15/30 | Train Loss: 0.3418 | Val Loss: 0.5496
Epoch 16/30 | Train Loss: 0.3347 | Val Loss: 0.5532
Epoch 17/30 | Train Loss: 0.3280 | Val Loss: 0.5576
Epoch 18/30 | Train Loss: 0.3218 | Val Loss: 0.5600
Epoch 19/30 | Train 

In [29]:
smooth = SmoothingFunction().method1

print("\n=== [추가실험 4-1] Seq2Seq (전처리) BLEU 평가 ===")
seq2seq_prep_bleu, sample_pairs_prep = evaluateRandomly_with_bleu(
    encoder_seq2seq_prep, decoder_seq2seq_prep,
    pairs_valid_prep,          # 전처리된 검증 쌍
    input_lang_prep, output_lang_prep,
    tokenizer_ko, tokenizer_en,
    n=5
)


=== [추가실험 4-1] Seq2Seq (전처리) BLEU 평가 ===
> KO: 반드시 제품 전원을 켠 상태에서 식물을 재배해 주십시오.
= EN: Make sure to grow plants with the product turned on.
< MT: Please feel free to use the product description of the product .
BLEU: 0.0428

> KO: 얼마나 회의를 연기하시기를 원하십니까?
= EN: How long do you want to postpone the meeting?
< MT: How much is the delivery date ?
BLEU: 0.0283

> KO: >느낌이 약간 유학파 같지 않아요?
= EN: Doesn't it feel like I studied abroad?
< MT: > Is n't it a lot of eggs , right ?
BLEU: 0.0441

> KO: 저희 식당에서는 최대 30명이 같이 식사할 수 있는 단체석이 있습니다.
= EN: In our restaurant, there are group seats where up to 30 people can eat together.
< MT: Our hostel is located here , so we can also be used for people who can feel the affection .
BLEU: 0.0136

> KO: 일정을 적절히 조정해 주세요.
= EN: Please adjust your schedule accordingly.
< MT: Please check the attached file .
BLEU: 0.0485

Average BLEU over 5 sentences: 0.0355


In [28]:
# 2) 전처리 버전 Seq2Seq + Attention
encoder_attn_prep = EncoderRNN(input_lang_prep.n_words, hidden_size).to(device)
decoder_attn_prep = AttnDecoderRNN(hidden_size, output_lang_prep.n_words).to(device)

print("\n=== [추가실험 4-2] Seq2Seq + Attention (전처리만 변경, MAX_LENGTH 유지) ===")
train_seq2seq(
    train_dataloader=train_dataloader_prep,
    valid_dataloader=valid_dataloader_prep,
    encoder=encoder_attn_prep,
    decoder=decoder_attn_prep,
    n_epochs=n_epochs,
    learning_rate=learning_rate,
    print_every=1
)


=== [추가실험 4-2] Seq2Seq + Attention (전처리만 변경, MAX_LENGTH 유지) ===
Epoch 1/30 | Train Loss: 0.7578 | Val Loss: 0.6077
Epoch 2/30 | Train Loss: 0.5688 | Val Loss: 0.5569
Epoch 3/30 | Train Loss: 0.5164 | Val Loss: 0.5275
Epoch 4/30 | Train Loss: 0.4779 | Val Loss: 0.5113
Epoch 5/30 | Train Loss: 0.4466 | Val Loss: 0.4999
Epoch 6/30 | Train Loss: 0.4205 | Val Loss: 0.4922
Epoch 7/30 | Train Loss: 0.3981 | Val Loss: 0.4881
Epoch 8/30 | Train Loss: 0.3781 | Val Loss: 0.4847
Epoch 9/30 | Train Loss: 0.3604 | Val Loss: 0.4835
Epoch 10/30 | Train Loss: 0.3444 | Val Loss: 0.4839
Epoch 11/30 | Train Loss: 0.3300 | Val Loss: 0.4829
Epoch 12/30 | Train Loss: 0.3169 | Val Loss: 0.4837
Epoch 13/30 | Train Loss: 0.3048 | Val Loss: 0.4836
Epoch 14/30 | Train Loss: 0.2941 | Val Loss: 0.4834
Epoch 15/30 | Train Loss: 0.2841 | Val Loss: 0.4854
Epoch 16/30 | Train Loss: 0.2748 | Val Loss: 0.4851
Epoch 17/30 | Train Loss: 0.2662 | Val Loss: 0.4863
Epoch 18/30 | Train Loss: 0.2582 | Val Loss: 0.4887
Epoch 19

In [30]:
print("\n=== [추가실험 4-2] Seq2Seq + Attention (전처리) BLEU 평가 ===")
attn_prep_bleu, _ = evaluateRandomly_with_bleu(
    encoder_attn_prep, decoder_attn_prep,
    sample_pairs_prep,         # 위에서 뽑은 동일 샘플들
    input_lang_prep, output_lang_prep,
    tokenizer_ko, tokenizer_en,
    n=len(sample_pairs_prep)
)


=== [추가실험 4-2] Seq2Seq + Attention (전처리) BLEU 평가 ===
> KO: 얼마나 회의를 연기하시기를 원하십니까?
= EN: How long do you want to postpone the meeting?
< MT: How many days do you want to change ?
BLEU: 0.2956

> KO: 일정을 적절히 조정해 주세요.
= EN: Please adjust your schedule accordingly.
< MT: Please calculate the schedule .
BLEU: 0.0579

> KO: 저희 식당에서는 최대 30명이 같이 식사할 수 있는 단체석이 있습니다.
= EN: In our restaurant, there are group seats where up to 30 people can eat together.
< MT: Our restaurant has a maximum of six people can eat up with a day .
BLEU: 0.0681

> KO: >느낌이 약간 유학파 같지 않아요?
= EN: Doesn't it feel like I studied abroad?
< MT: > Is it like a white way , but you 're eating it like this ?
BLEU: 0.0162

> KO: 반드시 제품 전원을 켠 상태에서 식물을 재배해 주십시오.
= EN: Make sure to grow plants with the product turned on.
< MT: Please use the power plant to make it easy to go .
BLEU: 0.0224

Average BLEU over 5 sentences: 0.0920


### 5.4. 가장 효과적이었던 설정 요약

- 실험별 손실함수 및 성능 지표

    | 실험 ID        | 모델                | hidden | lr     | EarlyStopping | 전처리 | 사용 에폭(실제) | Best Val Loss | Avg BLEU |
    | ------------ | ----------------- | ------ | ------ | ------------- | --- | --------- | ------------- | -------- |
    | 0-1 | Seq2Seq           | 128    | 0.001  | X             | X   | 30        | 0.5188        | 0.0586   |
    | 0-2    | Seq2Seq+Attention | 128    | 0.001  | X             | X   | 30        | 0.4887        | 0.0822   |
    | 1-1 | Seq2Seq           | 256    | 0.001  | X             | X   | 30        | 0.4996        | 0.0211   |
    | 1-2    | Seq2Seq+Attention | 256    | 0.001  | X             | X   | 30        | 0.4694        | 0.0480   |
    | 2-1 | Seq2Seq           | 128    | 0.0005 | X             | X   | 30        | 0.5289        | 0.0517   |
    | 2-2    | Seq2Seq+Attention | 128    | 0.0005 | X             | X   | 30        | 0.4860        | 0.0515   |
    | 3-1 | Seq2Seq           | 128    | 0.001  | O             | X   | 10        | 0.5317        | 0.1330   |
    | 3-2   | Seq2Seq+Attention | 128    | 0.001  | O             | X   | 16        | 0.4864        | 0.1355   |
    | 4-1 | Seq2Seq           | 128    | 0.001  | X             | O   | 30        | 0.5285        | 0.0355   |
    | 4-2   | Seq2Seq+Attention | 128    | 0.001  | X             | O   | 30        | 0.4829        | 0.0920   |


- 실험 결과를 보면, `실험 1-2(Seq2Seq+Attention, hidden=256)`는 Val Loss가 가장 낮아 손실 관점에서는 가장 좋은 값을 보였지만, BLEU는 오히려 `실험 0-2(Seq2Seq+Attention)`보다 낮아 번역 품질 측면에서는 아쉬움이 있었다.

- 반면 `실험 4-2(Seq2Seq+Attention, hidden=128, 전처리 적용)`는 Val Loss가 `실험 0-2`와 비슷한 수준을 유지하면서, BLEU가 0.0822(`실험 0-2`)에서 0.0920으로 상승해 정량적으로 가장 우수한 결과를 보였다.

- 정성 평가에서는 `실험 4-2`가 회의 연기, 일정 조정 등 문장에서 질문 구조와 핵심 의미를 비교적 잘 보존해, “조금 어색하지만 의도는 전달되는 번역”을 가장 많이 만들어 내는 설정으로 확인되었다.

- 종합적으로 고려했을 때, `실험 4-2`가 손실·BLEU·정성 평가를 모두 감안했을 때 가장 균형 잡힌 최종 후보로 보인다.

- 다만, 모든 실험이 GRU 기반 Seq2Seq/Attention 구조와 간단한 전처리, 단일 BLEU 지표에 한정되어 있어, 최신 Transformer 구조나 서브워드 토크나이저, 추가 평가 지표를 활용하지 못했다는 점에서 한계를 보인다.

- 이를 보완하기 위해, `실험 4-2 설정`을 기반으로 Early Stopping을 결합한 실험(`4-2 & 3-2`의 아이디어를 조합)이나, 동일 전처리·학습 설정을 Transformer 기반 번역 모델에 적용하는 추가 실험을 진행하면 더 나은 성능을 기대할 수 있다.

## 6. 결론  

### 6.1. 전체 실험 결과 요약 및 인사이트  

- 실험 `n-1`과 `n-2`를 비교했을 때, Attention을 사용한 실험 `n-2`가 항상 Seq2Seq인 `n-1`보다 낮은 Val Loss와 더 높은 BLEU를 보여, 구조 선택 측면에서 **Attention이 명확한 이점을 가진다**는 점이 확인되었다.
​
- 실험 `1-1`, `1-2`와 같이 hidden size를 256으로 키우면 손실 곡선 관점에서는 개선이 있었지만, BLEU는 실험 `0-2`보다 낮아져, **단순 용량 증가만으로는 번역 품질이 크게 향상되지 않는다**는 인사이트를 준다.
​
- 실험 `2-1`, `2-2`에서 학습률을 0.0005로 낮추면 훈련은 더 완만하게 수렴했지만, Best Val Loss와 BLEU는 실험 `0-1`, `0-2`와 비슷하거나 약간 낮은 수준에 머물러, **학습률 조정만으로 얻는 이득이 제한적임**을 보여 준다.
​
- 실험 `3-1`, `3-2`(Early Stopping)는 각각 10epoch, 16epoch에서 학습을 멈추면서도 Val Loss와 BLEU를 실험 `0-1`, `0-2`와 비슷한 수준으로 유지해, **성능을 크게 해치지 않고 학습 시간을 줄이는 전략**으로 유용함을 보였다.
​
- 실험 `4-1`, `4-2`(전처리 적용) 중 특히 `4-2`는 Val Loss는 실험 `0-2`와 유사하게 유지하면서 BLEU가 가장 높았고, 일정·회의 관련 문장에서 질문 구조와 의미를 더 잘 살리는 등 정성 평가에서도 상대적으로 우수해, **간단한 텍스트 정규화가 실제 번역 품질 개선에 효과적**이라는 점을 확인하게 해주었다.

### 6.2. 한계점 및 향후 개선 방향  

**한계점**
- 이번 프로젝트는 실험 `0-1`~`4-2`까지 모두 GRU 기반 Seq2Seq/Attention과 제한된 하이퍼파라미터 조합, 간단 전처리, 소규모 샘플에 대한 BLEU/정성 평가에 집중했다는 점에서, 모델 구조·토크나이즈 방식·평가 지표 측면의 탐색 범위가 제한적이라는 한계를 가진다.
​
- 또한 BLEU는 참조 번역과의 n-그램 중복에 민감해, 의미는 비슷하지만 표현이 다른 번역을 제대로 평가하지 못하는 경우가 있어, 실험 `3-1`, `3-2`처럼 일부 짧은 문장에서 BLEU가 비정상적으로 높게 나오는 사례도 관찰되었다.

**개선 방향**
- 실험 `4-2`를 기반으로 실험 `3-2`의 Early Stopping 전략을 결합한 **“전처리+Attention+ES”** 실험을 추가해, 시간 대비 성능 효율을 더 끌어올릴 수 있다.

- SentencePiece/BPE 기반 서브워드 토크나이저를 도입하고, Transformer 기반 모델을 적용하는 방향을 고려할 수 있다.

- 더 큰 검증 세트와 추가 지표(METEOR, chrF 등)를 활용한 평가를 통해, 현재 실험 `0-2`, `3-2`, `4-2`에서 얻은 인사이트를 넘어서는 번역 품질 향상을 단계적으로 시도해 볼 수 있다.